In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [3]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [5]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
y,tx = build_model_data(tx,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [6]:
num_samples, num_features
tx.shape

(250000, 24)

In [7]:
from compute_gradient import *
from cost import *
from implementations import *

In [8]:
#Step-size

lambdas = np.logspace (-20,-1,20)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 24), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [9]:
losses_tr = np.zeros(len(lambdas))
losses_val = np.zeros(len(lambdas))

k_fold = 10
seed = 1
loss_min = np.inf
max_accuracy = 0

#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    
    loss_training = 0
    loss_validation = 0
    training_accuracy = 0
    validation_accuracy = 0
    for i,k in enumerate(range(k_fold)):
        tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
        w, loss_train_k = ridge_regression(y_train, tx_train, lambda_)
        loss_val_k = ridge_loss (y_val,tx_val,w,lambda_)
        loss_training += loss_train_k
        loss_validation += loss_val_k
        training_accuracy += predict_accuracy(y_train,tx_train,w)
        validation_accuracy += predict_accuracy(y_val,tx_val,w)
        
    training_accuracy /= k_fold
    validation_accuracy /= k_fold
    loss_training /= k_fold
    loss_validation /= k_fold
    print(lambda_)
    print(loss_training)
    print(loss_validation)
    print(training_accuracy)
    print(validation_accuracy)
    if (validation_accuracy > max_accuracy):
        weights_star = w
        lambda_star = lambda_
        loss_min = loss_validation
        max_accuracy = validation_accuracy 
    losses_tr[ind_lambda] = loss_training
    losses_val[ind_lambda] = loss_validation



1e-20
0.08804022443947605
0.08806169627305069
73.37386666666666
73.36279999999998
1e-19
0.08804022443947605
0.08806169627305704
73.37386666666666
73.36279999999998
1e-18
0.08804022443947605
0.08806169627312052
73.37386666666666
73.36279999999998
1e-17
0.08804022443947605
0.0880616962737554
73.37386666666666
73.36279999999998
1e-16
0.08804022443947652
0.08806169626244842
73.37386666666666
73.36279999999998
1e-15
0.08804022443948857
0.08806169619539159
73.37386666666666
73.36279999999998
1e-14
0.08804022444062534
0.08806169549608875
73.37395555555555
73.36319999999998
1e-13
0.08804022454902435
0.08806168860809871
73.37440000000001
73.36399999999999
1e-12
0.08804023283302012
0.08806162905145751
73.37506666666667
73.36280000000002
1e-11
0.08804039137725453
0.08806139556004147
73.37804444444444
73.3604
1e-10
0.0880406147439882
0.08806123551833664
73.37862222222222
73.3672
1e-09
0.0880406595683108
0.08806120979696085
73.37862222222222
73.36639999999998
1e-08
0.08804066447078959
0.08806120887

In [24]:
lambdas

array([1.00000000e-06, 6.81292069e-06, 4.64158883e-05, 3.16227766e-04,
       2.15443469e-03, 1.46779927e-02, 1.00000000e-01])

In [29]:
loss_min, lambda_star, max_accuracy

(nan, 0.1, 0.014677992676220705, 66.44239999999999)

In [30]:
#PREDICT TRAINING ACCURACY
training_accuracy = predict_accuracy(y,tx,weights_star)
training_accuracy

58.6948

In [198]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [203]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [206]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [207]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [210]:
tx_test_augmented = build_poly_all_features(tx_test_clean,4)

(568238, 23)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre5.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)